#Fine-tune AraBERT with Fast-BERT library
https://github.com/kaushaltrivedi/fast-ber

In [1]:
!git clone https://github.com/aub-mind/arabert
!pip install PyArabic farasapy fast-bert

Cloning into 'arabert'...
remote: Enumerating objects: 544, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 544 (delta 175), reused 237 (delta 87), pack-reused 214
Receiving objects: 100% (544/544), 9.15 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (298/298), done.
     |████████████████████████████████| 126 kB 12.9 MB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
     |████████████████████████████████| 3.0 MB 23.0 MB/s 
     |████████████████████████████████| 124 kB 52.3 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 769 kB 35.3 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 1.2 MB 19.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=a4c567f843e5182c29d3ba39dc7b7b6ff18da458c2cb1fedec22b25ac7bebf2e
  Stored in directory: /root/.cache/pip/wheels/0

In [2]:
!pip install emoji
import emoji
import regex 
import pandas as pd
import os

def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

     |████████████████████████████████| 185 kB 11.3 MB/s 
  Created wheel for emoji: filename=emoji-1.5.0-py3-none-any.whl size=187457 sha256=4dd620c0463056ca7318d80fdbe637176b231afb1a011d25ff852487e15360e1
  Stored in directory: /root/.cache/pip/wheels/db/b5/f6/b39abf14e94b3d6640613bbe630a66c10ccf7a12882d064fb5
Successfully built emoji


In [3]:
import re


text = []
type_of_emotion = []

file1 = open('/content/SemEval2017-task4-train.subtask-A.arabic.txt','r')
data= file1.readlines()

for j in range(1,len(data)):
  dt = data[j].split('\t')
  line = dt[2]
  emojis = split_count(line)
    
  for em in emojis : 
    line = re.sub(em,'',line)
  
  line = re.sub('[0-9@]', '',line)
  line = re.sub('[a-zA-z]', '',line)
  line = re.sub('[#[]', '',line)
  line = re.sub(']', '',line)
  line = line.rstrip().lstrip()
  text.append(line)
  type_of_emotion.append(dt[1])



In [4]:
from farasa.segmenter import FarasaSegmenter
from arabert.preprocess import ArabertPreprocessor
farasa_segmenter = FarasaSegmenter(interactive=True)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:12<00:00, 18.8MiB/s]


[2021-09-21 19:54:52,397 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [6]:
train_data = pd.DataFrame(data={'text':text,'label':type_of_emotion})
segmenter = FarasaSegmenter(interactive=True)
train_data['text'] = train_data['text'].apply(lambda x: segmenter.segment(x) )
train_data.head()

#train_data['text'] = train_data['text'].apply(lambda x: pre.preprocess(x, farasa=farasa_segmenter, use_farasapy = True) )
#train_data.head()

[2021-09-21 19:55:21,553 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


,text,label
0,""" : غوغل تتحدى أبل و+أمازون ب+أجهز+ة جديد+ة : ...",positive
1,""" جوجل تنافس أبل و+سامسونج ب+هاتف جديد ://./""",positive
2,""" رئيس شرك+ة أبل : ال+واقع ال+معزز س+يصبح أهم ...",positive
3,""" ساع+ة أبل في ال+أسواق مر+ة أخرى ://./""",neutral
4,""" أبل و+ال+جسار تفقد+ا مدين+ة « صباح ال+أحمد »...",positive


In [7]:
!mkdir data

In [8]:
train_data.to_csv('data/train.csv',index=False)

In [9]:
text = []
type_of_emotion = []

file1 = open('/content/SemEval2017-task4-test.subtask-A.arabic.txt','r')
data= file1.readlines()

for j in range(1,len(data)):
  dt = data[j].split('\t')
  line = dt[2]
  emojis = split_count(line)
    
  for em in emojis : 
    line = re.sub(em,'',line)
  
  line = re.sub('[0-9@]', '',line)
  line = re.sub('[a-zA-z]', '',line)
  line = re.sub('[#[]', '',line)
  line = re.sub(']', '',line)
  line = line.rstrip().lstrip()
  text.append(line)
  type_of_emotion.append(dt[1])
test_data = pd.DataFrame(data={'text':text,'label':type_of_emotion})
segmenter = FarasaSegmenter(interactive=True)
test_data['text'] = test_data['text'].apply(lambda x: segmenter.segment(x) )
test_data.head()


[2021-09-21 19:55:37,732 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


,text,label
0,""" كلام سليم """,UNKNOWN
1,""" ياخزياه بس صارخ+ت صراخ و+ازعج+ت+كم مشكل+ة ال...",UNKNOWN
2,""" صوتي راح اقولك ? ? """,UNKNOWN
3,""" أنا ماخفت من شي الا ان يكون عقلي+ة مشجع+ي ال...",UNKNOWN
4,""" ب+دخل ال+كلاس ب+طريق+ة احتفال كريستيانو ://./""",UNKNOWN


In [10]:
test_data.to_csv('data/test.csv',index=False)

In [11]:
text = []
type_of_emotion = []

file1 = open('/content/SemEval2017-task4-dev.subtask-A.arabic.INPUT VALIDATION.txt','r')
data= file1.readlines()

for j in range(1,len(data)):
  dt = data[j].split('\t')
  line = dt[2]
  emojis = split_count(line)
    
  for em in emojis : 
    line = re.sub(em,'',line)
  
  line = re.sub('[0-9@]', '',line)
  line = re.sub('[a-zA-z]', '',line)
  line = re.sub('[0-9@]', '',line)
  line = re.sub('[a-zA-z]', '',line)
  line = re.sub('[#[]', '',line)
  line = re.sub(']', '',line)
  line = line.rstrip().lstrip()
  text.append(line)
  type_of_emotion.append(dt[1])
val_data = pd.DataFrame(data={'text':text,'label':type_of_emotion})
segmenter = FarasaSegmenter(interactive=True)
val_data['text'] = val_data['text'].apply(lambda x: segmenter.segment(x) )
val_data.head()


[2021-09-21 19:55:56,196 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


,text,label
0,""" أخر صيح+ة من حقائب ال+يد من ف+ندي و+غوتشي :/...",neutral
1,""" تتشرف باريغاليري ب+دعو+ت+كم ل+حضور و+مقابل+ة...",positive
2,""" : ستايلبنطال نامج+ون ال+جينز ل+صور كان ب+سعر...",neutral
3,""" : كريس في ازياء كريس جميع+ها من بربري من ال+...",neutral
4,""" تعرف+ي على حذاء غوتشي الذي يجمع بين ال+كعب ا...",neutral


In [12]:
val_data.to_csv('data/dev.csv',index=False)

##Get Requirements

##Prepare the data

In [13]:
import pandas as pd

label_map = {'positive':1,'negative':-1,'neutral':0}

with open('data/labels.csv','w') as f:
  f.write("\n".join(train_data['label'].unique()))

##Create a DataBunch Object:
see https://github.com/kaushaltrivedi/fast-bert#text-classification

In [14]:
from fast_bert.data_cls import BertDataBunch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')

databunch = BertDataBunch('./data/', './data/',
                          tokenizer=tokenizer,
                          train_file='train.csv',
                          val_file='dev.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col='label',
                          batch_size_per_gpu=16,
                          max_seq_length=256,
                          multi_gpu=True,
                          multi_label= False,
                          model_type='bert',
                          )

Downloading:   0%|          | 0.00/572 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/717k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

##Create the Learner Object
see https://github.com/kaushaltrivedi/fast-bert#2-create-a-learner-object

In [15]:
import logging
import torch

from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

!mkdir 'output'
learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='aubmindlab/bert-base-arabert',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir='output',
						finetuned_wgts_path=None,
						warmup_steps=30,
						multi_gpu=False,
						is_fp16=False,
						multi_label= False,
						logging_steps=0)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/aubmindlab/bert-base-arabert/config.json from cache at /root/.cache/torch/transformers/91c3e98e149f6e88215bffd705e4ef9bd8a355f4c317973e4f3868c6f93fa24a.87c61215f57298a5ff1f7680910adeb70154f00bf4e7a0fe7d5ab21ee115c7cf
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "poole

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

INFO:transformers.file_utils:storing https://cdn.huggingface.co/aubmindlab/bert-base-arabert/pytorch_model.bin in cache at /root/.cache/torch/transformers/26f8720a79d80f7ae70d448d93e09fed364a6b0393c95a026519402dbd1313b2.8867d4172d26dd413ba8029e13bbe7eb5b9ae301b62c259f8be84ee78c0beb0c
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/26f8720a79d80f7ae70d448d93e09fed364a6b0393c95a026519402dbd1313b2.8867d4172d26dd413ba8029e13bbe7eb5b9ae301b62c259f8be84ee78c0beb0c
INFO:filelock:Lock 140621832310480 released on /root/.cache/torch/transformers/26f8720a79d80f7ae70d448d93e09fed364a6b0393c95a026519402dbd1313b2.8867d4172d26dd413ba8029e13bbe7eb5b9ae301b62c259f8be84ee78c0beb0c.lock
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/aubmindlab/bert-base-arabert/pytorch_model.bin from cache at /root/.cache/torch/transformers/26f8720a79d80f7ae70d448d93e09fed364a6b0393c95a026519402dbd1313b2.8867d4172d26dd413ba8029e13bbe7eb5b9ae301b62c

##Start Training and Validating

In [16]:
learner.fit(epochs=5,
			lr=2e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_linear",
			optimizer_type="adamw")

INFO:__main__:***** Running training *****
INFO:__main__:  Num examples = 3354
INFO:__main__:  Num Epochs = 5
INFO:__main__:  Total train batch size (w. parallel, distributed & accumulation) = 16
INFO:__main__:  Gradient Accumulation steps = 1
INFO:__main__:  Total optimization steps = 1050


INFO:__main__:Running evaluation
INFO:__main__:  Num examples = 670
INFO:__main__:  Batch size = 32


INFO:__main__:eval_loss after epoch 1: 0.6244760354359945: 
INFO:__main__:eval_accuracy after epoch 1: 0.7298507462686568: 
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
INFO:__main__:lr after epoch 1: 1.647058823529412e-05
INFO:__main__:train_loss after epoch 1: 0.8897402505079905
INFO:__main__:

INFO:__main__:Running evaluation
INFO:__main__:  Num examples = 670
INFO:__main__:  Batch size = 32


INFO:__main__:eval_loss after epoch 2: 0.41793719146932873: 
INFO:__main__:eval_accuracy after epoch 2: 0.8507462686567164: 
INFO:__main__:lr after epoch 2: 1.235294117647059e-05
INFO:__main__:train_loss after epoch 2: 0.6292213095085961
INFO:__main__:

INFO:__main__:Running evaluation
INFO:__main__:  Num examples = 670
INFO:__main__:  Batch size = 32


INFO:__main__:eval_loss after epoch 3: 0.22418740143378577: 
INFO:__main__:eval_accuracy after epoch 3: 0.926865671641791: 
INFO:__main__:lr after epoch 3: 8.23529411764706e-06
INFO:__main__:train_loss after epoch 3: 0.41562566845899535
INFO:__main__:

INFO:__main__:Running evaluation
INFO:__main__:  Num examples = 670
INFO:__main__:  Batch size = 32


INFO:__main__:eval_loss after epoch 4: 0.11558223657664798: 
INFO:__main__:eval_accuracy after epoch 4: 0.9641791044776119: 
INFO:__main__:lr after epoch 4: 4.11764705882353e-06
INFO:__main__:train_loss after epoch 4: 0.26979370376183875
INFO:__main__:

INFO:__main__:Running evaluation
INFO:__main__:  Num examples = 670
INFO:__main__:  Batch size = 32


INFO:__main__:eval_loss after epoch 5: 0.08530548897882302: 
INFO:__main__:eval_accuracy after epoch 5: 0.9746268656716418: 
INFO:__main__:lr after epoch 5: 0.0
INFO:__main__:train_loss after epoch 5: 0.18555972832476808
INFO:__main__:



(1050, 0.4779881321124378)

In [19]:
learner.save_model()

INFO:transformers.configuration_utils:Configuration saved in output/model_out/config.json
INFO:transformers.modeling_utils:Model weights saved in output/model_out/pytorch_model.bin


In [20]:
from fast_bert.prediction import BertClassificationPredictor

predictor = BertClassificationPredictor(model_path='output/model_out', 
                                        label_path='data', multi_label=False)


INFO:transformers.configuration_utils:loading configuration file output/model_out/config.json
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 64000
}

INFO:transformer

In [21]:
predictor.predict(val_data.text[1]) , val_data.label[1]

INFO:root:Writing example 0 of 1
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


([('positive', 0.9819854497909546),
  ('neutral', 0.016282830387353897),
  ('negative', 0.0017317425226792693)],
 'positive')

In [22]:
pred = []
true = []

for k in range(len(val_data.text)):
  prd = predictor.predict(val_data.text[k])
  true.append(label_map[val_data.label[k]])
  
  tab = [prd[0][1],prd[1][1],prd[2][1]]
  tab
  max = tab[0]
  indice = prd[0][0]
  for j in range(1,3):
    if tab[j]>max :
      max = tab[j]
      indice = prd[j][0]
  max , indice
  
  
  pred.append(label_map[indice])

INFO:root:Writing example 0 of 1
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Writing example 0 of 1
INFO:root:Wr

In [23]:
val_data['pred arab'] = pred
val_data.head()

,text,label,pred arab
0,""" أخر صيح+ة من حقائب ال+يد من ف+ندي و+غوتشي :/...",neutral,0
1,""" تتشرف باريغاليري ب+دعو+ت+كم ل+حضور و+مقابل+ة...",positive,1
2,""" : ستايلبنطال نامج+ون ال+جينز ل+صور كان ب+سعر...",neutral,0
3,""" : كريس في ازياء كريس جميع+ها من بربري من ال+...",neutral,0
4,""" تعرف+ي على حذاء غوتشي الذي يجمع بين ال+كعب ا...",neutral,0


In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')


INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/aubmindlab/bert-base-arabert/config.json from cache at /root/.cache/torch/transformers/91c3e98e149f6e88215bffd705e4ef9bd8a355f4c317973e4f3868c6f93fa24a.87c61215f57298a5ff1f7680910adeb70154f00bf4e7a0fe7d5ab21ee115c7cf
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",

In [25]:
val_data['text tok'] = val_data['text'].apply(lambda x: tokenizer.tokenize(x) )
val_data.head()

,text,label,pred arab,text tok
0,""" أخر صيح+ة من حقائب ال+يد من ف+ندي و+غوتشي :/...",neutral,0,"["", أخر, صيح, +, ة, من, حقائب, ال, +, يد, من, ..."
1,""" تتشرف باريغاليري ب+دعو+ت+كم ل+حضور و+مقابل+ة...",positive,1,"["", تتش, ##رف, باري, ##غالي, ##ري, ب, +, دعو, ..."
2,""" : ستايلبنطال نامج+ون ال+جينز ل+صور كان ب+سعر...",neutral,0,"["", :, ستايل, ##بن, ##طال, نام, ##ج, +, و, ##ن..."
3,""" : كريس في ازياء كريس جميع+ها من بربري من ال+...",neutral,0,"["", :, كريس, في, ازيا, ##ء, كريس, جميع, +, ه, ..."
4,""" تعرف+ي على حذاء غوتشي الذي يجمع بين ال+كعب ا...",neutral,0,"["", تعرف, +, ي, على, حذاء, غوتشي, الذي, يجمع, ..."


## read the lexicon data

In [26]:
f = open('/content/SauDiSenti lexicon.txt','r')

lines = f.readlines()

dic= {} # key type(neg,pos) -> value list of words
dic['pos'] = []
dic['neg'] = []

for ln in lines:

  tp = int(ln.split('\t')[1])
  if tp ==1:
    dic['pos'].append(ln.split('\t')[0].replace('\n',''))
  else:
    dic['neg'].append(ln.split('\t')[0].replace('\n',''))
 
  
  
  

In [27]:
len(dic['pos']) , len(dic['neg'])

(1048, 3220)

In [28]:
pred = []

for index,row in val_data.iterrows():
  text = list(row['text tok'])
  

  inters_pos = set(text).intersection(set(dic['pos']))
  inters_neg = set(text).intersection(set(dic['neg']))
  

  if len(inters_neg)==0 and len(inters_pos)==0:
    pred.append('neutral')
  else:
    decide = (len(inters_pos)-len(inters_neg))/(len(inters_pos)+len(inters_neg))
    

    if decide == 0:
      pred.append('neutral')
    elif decide>0:
      pred.append('positive') 
    else:
      pred.append('negative')

In [29]:
val_data['pred lexicon'] = pred

In [30]:
val_data['pred_lexicon_int'] = val_data['pred lexicon'].apply(lambda x: label_map[x])
val_data['label_int'] = val_data['label'].apply(lambda x: label_map[x])

val_data.head()

,text,label,pred arab,text tok,pred lexicon,pred_lexicon_int,label_int
0,""" أخر صيح+ة من حقائب ال+يد من ف+ندي و+غوتشي :/...",neutral,0,"["", أخر, صيح, +, ة, من, حقائب, ال, +, يد, من, ...",neutral,0,0
1,""" تتشرف باريغاليري ب+دعو+ت+كم ل+حضور و+مقابل+ة...",positive,1,"["", تتش, ##رف, باري, ##غالي, ##ري, ب, +, دعو, ...",neutral,0,1
2,""" : ستايلبنطال نامج+ون ال+جينز ل+صور كان ب+سعر...",neutral,0,"["", :, ستايل, ##بن, ##طال, نام, ##ج, +, و, ##ن...",neutral,0,0
3,""" : كريس في ازياء كريس جميع+ها من بربري من ال+...",neutral,0,"["", :, كريس, في, ازيا, ##ء, كريس, جميع, +, ه, ...",neutral,0,0
4,""" تعرف+ي على حذاء غوتشي الذي يجمع بين ال+كعب ا...",neutral,0,"["", تعرف, +, ي, على, حذاء, غوتشي, الذي, يجمع, ...",neutral,0,0


In [31]:
from sklearn.metrics import accuracy_score as acc
print('acc with lexicon',acc(val_data['label_int'],val_data['pred_lexicon_int']))
print('acc with arabert',acc(val_data['label_int'],val_data['pred arab']))


acc with lexicon 0.5208955223880597
acc with arabert 0.9746268656716418
